https://towardsdatascience.com/data-science-skills-web-scraping-javascript-using-python-97a29738353f

https://exceptionshub.com/selenium-cant-click-element-because-other-element-obscures-it.html

https://www.toolsqa.com/selenium-webdriver/handle-iframes-in-selenium/


In [26]:
import time, datetime, requests,random


from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver

import pandas as pd

#//*********************************************************
#//*** Build the tgt_date from the slected Quarter and Year
#//*********************************************************
tgt_year = 2021
quarter = "Q4"

if quarter == "Q1":
    tgt_date = f"{tgt_year}-01-01"
elif quarter == "Q2": 
    tgt_date = f"{tgt_year}-04-01"
elif quarter == "Q3": 
    tgt_date = f"{tgt_year}-07-01"
elif quarter == "Q4": 
    tgt_date = f"{tgt_year}-10-01"

tgt_date = datetime.datetime.strptime(tgt_date, "%Y-%m-%d")

print(tgt_date)



2021-10-01 00:00:00
1.352


In [103]:
#//*** Parses a 
def get_scripts(source_urls,headless=True):
    
    out = []
    
    #//*** Initialize Headless Firefox options 
    options = webdriver.FirefoxOptions()
    options.add_argument('-headless')   
    
    if headless:


        #//*** Span Headless Instance
        page_driver = webdriver.Firefox(options=options)
    else:
        # run firefox webdriver from executable path of your choice
        page_driver = webdriver.Firefox()

    
    
    for url in source_urls:
    
    
        page_driver.get(url)
        
        loop_story = {
            "headline" : page_driver.find_elements_by_class_name("headline")[0].text,
            "time_text" : page_driver.find_elements_by_tag_name("time")[0].text,
            "body_text" : page_driver.find_elements_by_class_name("body-text")[0].text,
        }
        #//*** Split the Date
        loop_story['time_date'] = loop_story['time_text'].split()
        
        #//*** Rebuild Date string using only the first four fields. Drop the extra time
        loop_story['time_date'] = " ".join(loop_story['time_date'][:4])
        
        loop_story['time_date'] = datetime.datetime.strptime(loop_story['time_date'], "%A, %B %d, %Y")
        
        out.append(loop_story)
        
        #wait between 1/2 to 3 seconds
        time.sleep(random.randint(500, 3000)/1000)                                                              

    page_driver.quit()
                                                                        
    return out
    
    
def get_all_urls_in_page(source_url,tgt_date,headless=True):

    #//*** Keep all classnames in a dictionary
    g = {
        #//*** Classname for the Show More Button
        "class_more_button" : "show-button-more",
        
        #//*** Grid Classname that contains the linked stories
        "story_grid" : "grid3",
        
        #//*** Story Classes that contain the href Links
        "story_link_class" : "AnchorLink",
        
    }
    
    #//*** Initialize Headless Firefox options 
    options = webdriver.FirefoxOptions()
    options.add_argument('-headless')   
    
    if headless:


        #//*** Span Headless Instance
        driver = webdriver.Firefox(options=options)
    else:
        # run firefox webdriver from executable path of your choice
        driver = webdriver.Firefox()


    # Load Web Page
    driver.get(source_url)
    
    # execute script to scroll down the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")


    #//*****************************************************
    #//*** Wait until the Show More Button is loaded
    #//*****************************************************
    elements = []
    control = 0
    while len(elements) == 0:

        #//*** Page is Loaded when class_more_button is displayed
        elements = driver.find_elements_by_class_name(g["class_more_button"])

        control += 1
        #//*** Wait if element not found
        if len(elements) == 0:
            time.sleep(1)

        if control > 15:
            print("Show More Button Not Loaded!")
            return
            break

    #//*************************************************
    #//*** The first elements is the button to click
    #//*************************************************
    elem_show_more = elements[0]

    #actions = ActionChains(driver)
    #actions.move_to_element(element).perform()

    print(elem_show_more.is_displayed())
    print(elem_show_more.location['y'])
    
    results = driver.find_elements_by_class_name(g["story_grid"])[0].find_elements_by_class_name(g["story_link_class"])

    if len(results) > 0:
        print("Last Page: ", results[0].get_property('href'))

    
    
    #print(results[-1].get_property('href'))

    
    #print(story_time,tgt_date, story_time < tgt_date)
                                                                        
    #//*** Get the last url to check in stories are within the quarter
    last_url = results[-1].get_property('href')
    
    #//*** Download the last script
    last_script = get_scripts([last_url])[0]
    last_date = last_script['time_date']
    print("Last Script: ", last_script['headline'],"\n",last_script["time_text"])
    print("Last Script: ", last_script['time_date'] < tgt_date)
    
    #//*** Maximum number of while loops.
    maxDepth = 10
    depth = 0
    
    #//*** Click Show More until Last Story is older than the tgt_date
    while last_date > tgt_date:
        depth += 1
        
        #//*** Scroll to Bottom of page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        
        print(f"Clicking More Depth: {depth}")
        #print(dir(elem_show_more))
        while elem_show_more.is_displayed() == False:
            print("Is Displayed:",str(elem_show_more.is_displayed()))
            time.sleep(1)
        
        
        try:
            #//*** Click Show More
            elem_show_more.click()
        except:
            iframes = driver.find_elements_by_tag_name("iframe")
            for iframe in iframes:
                print(iframe.get_property("name"))
                
                
                driver.execute_script("arguments[0].style.visibility='hidden'", iframe)
            #//*** Click Show More
            elem_show_more.click()
                
            return

    
        #//*** Get Updated Results
        results = driver.find_elements_by_class_name(g["story_grid"])[0].find_elements_by_class_name(g["story_link_class"])        
        
        #//*** Get the last url to check in stories are within the quarter
        last_url = results[-1].get_property('href')

        #//*** Download the last script
        last_script = get_scripts([last_url])[0]
        last_date = last_script['time_date']
        print("Last Script: ", last_script['headline'],"\n",last_script["time_text"])
        print("Last Script: ", last_script['time_date'] < tgt_date)

        if depth > maxDepth:
            print("Maximum Depth Reached on Clicking Show More")
            break
        
    
    out = []
    
    for result in results:
        out.append(result.get_property('href'))
    
    #//*** Shut down main scrape
    driver.quit()
    
    

    return out

urlpages = ['https://abc7news.com/education/',
            'https://abc7news.com/tag/building-a-better-bay-area/',
            'https://abc7news.com/7onyourside/',
            'https://abc7news.com/tag/covid-19/',
            'https://abc7news.com/tag/climate-change/',
            'https://abc7news.com/iteam/'
           ]

urlpage ='https://abc7news.com/education/'
urls = get_all_urls_in_page(urlpage,tgt_date,True)
print(urlpage)


True
1848
Last Page:  https://abc7news.com/oakland-school-for-the-arts-osa-emergency-closure-covid-cases/11452106/
Last Script:  California State University to require COVID-19 booster shots for spring semester 
 Wednesday, December 22, 2021
Last Script:  False
Clicking More Depth: 1
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Is Displayed: False
Last Script:  California State University to require COVID-19 booster shots for spring semester 
 Wednesday, December 22, 2021
Last Script:  False
Clicking More Depth: 2
Last Script:  'Femineer' club offers STEM program to help girls excel at Bay Area school 
 Thursday, December 2, 2021
Last Script:  False
Clicking More Depth: 3
Last Script:  SFUSD proposes difficult spending cuts to avoid state takeover 
 Tuesday, November 2, 2021
Last Script:  False
Clicking More Depth: 4
Last Script:  Su

In [96]:
driver = webdriver.Firefox()

        
dir(driver)

['CONTEXT_CHROME',
 'CONTEXT_CONTENT',
 'NATIVE_EVENTS_ALLOWED',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_file_detector',
 '_is_remote',
 '_mobile',
 '_switch_to',
 '_unwrap_value',
 '_web_element_cls',
 '_wrap_value',
 'add_cookie',
 'application_cache',
 'back',
 'binary',
 'capabilities',
 'close',
 'command_executor',
 'context',
 'create_web_element',
 'current_url',
 'current_window_handle',
 'delete_all_cookies',
 'delete_cookie',
 'desired_capabilities',
 'error_handler',
 'execute',
 'execute_async_script',
 'execute_script',
 'file_detector',
 'file_detector_context',
 'find_element',
 'find_element_by_class_name',
 'find_e

In [87]:


scripts = get_scripts(urls)

KeyboardInterrupt: 

In [60]:
t = "Monday, January 10, 2022 11:18PM".split(" ")
t = " ".join(t[:4])
print(t)

Monday, January 10, 2022


In [62]:
for script in scripts:
    print(script)

{'headline': 'Oakland School for the Arts issues emergency COVID closure for rest of this week', 'time_text': 'Monday, January 10, 2022 11:18PM', 'body_text': 'OAKLAND, Calif. (KGO) -- The surge of coronavirus cases is having a major impact on classrooms -- So much so, that one school instituted an emergency closure for the rest of the week.\n\nIt\'s the Oakland School for the Arts, which is a nonprofit charter school, and not part of the Oakland Unified School District.\n\n"During our 7th period class teachers told us that our school would be closing for the rest of the week," said 8th grader Jayla Brown.\n\nRELATED: UC Berkeley students push for remote start to spring semester amid COVID surge\n\nThat\'s because 20 teachers at Oakland School for the Arts are out with COVID-19 or out after having been exposed to the virus.\n\n"Two of my teachers were out today," said 7th grader Micah Head.\n\n"My Art teacher was gone the whole last week, and my math teacher was gone, and my English te

In [33]:
elem_show_more.click()
#driver.implicitly_wait(1)

#driver.execute_script("arguments[0].click();", element)
#print(element.is_displayed())


In [13]:
dir(driver)

['CONTEXT_CHROME',
 'CONTEXT_CONTENT',
 'NATIVE_EVENTS_ALLOWED',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_file_detector',
 '_is_remote',
 '_mobile',
 '_switch_to',
 '_unwrap_value',
 '_web_element_cls',
 '_wrap_value',
 'add_cookie',
 'application_cache',
 'back',
 'binary',
 'capabilities',
 'close',
 'command_executor',
 'context',
 'create_web_element',
 'current_url',
 'current_window_handle',
 'delete_all_cookies',
 'delete_cookie',
 'desired_capabilities',
 'error_handler',
 'execute',
 'execute_async_script',
 'execute_script',
 'file_detector',
 'file_detector_context',
 'find_element',
 'find_element_by_class_name',
 'find_e

In [81]:
#//*** Cache Scripts to a DataFrame
df = pd.DataFrame()
df['urls'] = urls
df['hash'] = pd.util.hash_pandas_object(df['urls'])
df['cat'] = 'Education'

s = {}

for script in scripts:
    
    for key,value in script.items():
        if key not in s.keys():
            s[key] = []
        s[key].append(value)
    

for key,value in s.items():
    df[key] = value

filename = f"{tgt_year}_{quarter}_scripts.dat"

pd.to_pickle(df,filename)


In [83]:
tdf =  pd.read_pickle(filename)

tdf

,urls,hash,cat,headline,time_text,body_text,time_date
0,https://abc7news.com/oakland-school-for-the-ar...,13209601780715947492,Education,Oakland School for the Arts issues emergency C...,"Monday, January 10, 2022 11:18PM","OAKLAND, Calif. (KGO) -- The surge of coronavi...",2022-01-10
1,https://abc7news.com/child-care-covid-day-test...,12909625083784510628,Education,Some Bay Area child care centers close due to ...,"Monday, January 10, 2022 7:03PM","OAKLAND, Calif. (KGO) -- Several child care ce...",2022-01-10
2,https://abc7news.com/school-covid-virtual-lear...,2850493963882229434,Education,'A decade behind': CA official explains what s...,"Saturday, January 8, 2022 11:48PM",SAN FRANCISCO (KGO) -- There is growing concer...,2022-01-08
3,https://abc7news.com/musd-milpitas-remote-lear...,16105722897121743005,Education,Milpitas Unified to keep in-person learning de...,"Saturday, January 8, 2022 6:10PM","MILPITAS, Calif. (KGO) -- UPDATE: Milpitas Uni...",2022-01-08
4,https://abc7news.com/chinatown-voting-right-im...,17178305827214377954,Education,"SF Standard discusses immigrant voting rights,...","Friday, January 7, 2022 5:09PM",SAN FRANCISCO (KGO) -- ABC7 is partnering with...,2022-01-07
...,...,...,...,...,...,...,...
67,https://abc7news.com/lincoln-university-viral-...,6454552566090952546,Education,America Strong: Lincoln University professor g...,"Tuesday, September 14, 2021","LINCOLN UNIVERSITY, Pennsylvania -- The story ...",2021-09-14
68,https://abc7news.com/bus-driver-shortage-2021-...,13336946352295937915,Education,Massachusetts taps National Guard to help scho...,"Tuesday, September 14, 2021",BOSTON -- As school districts across the U.S. ...,2021-09-14
69,https://abc7news.com/philippe-croizon-amutee-f...,16044220988261636096,Education,'Anything is possible': Athlete and quadruple ...,"Monday, September 13, 2021",SAN FRANCISCO (KGO) -- In these times of conti...,2021-09-13
70,https://abc7news.com/education-malcolm-x-acade...,2253032003515501374,Education,Equity in the Classroom: Black teacher in SF's...,"Monday, September 13, 2021","SAN FRANCISCO (KGO) -- In the last 18 months, ...",2021-09-13


In [35]:
results = driver.find_elements_by_class_name("grid3")[0].find_elements_by_class_name("AnchorLink")

if len(results) > 0:
    print("Last Page: ", results[0].get_property('href'))
for result in results:
    print(result.get_property('href'))

Last Page:  https://abc7news.com/chinatown-voting-right-immigrants-school-board-recall-election-sf-standard/11438958/
https://abc7news.com/chinatown-voting-right-immigrants-school-board-recall-election-sf-standard/11438958/
https://abc7news.com/teacher-dancing-tiktok-youre-a-jerk-dance-viral-tenaya-middle-school/11438306/
https://abc7news.com/uc-berkeley-spring-semester-covid-cases-students-push-for-remote-learning/11435743/
https://abc7news.com/chicago-public-schools-cancels-classes-remote-learning-covid-19-cases/11427768/
https://abc7news.com/abbott-elementary-abc-new-show-hulu-shows/11425882/
https://abc7news.com/covid-testing-sfusd-ousd-back-to-school-tests/11422095/
https://abc7news.com/bus-driver-shortage-marin-school-buses-horizon-county/11404507/
https://abc7news.com/california-state-university-covid-19-booster-csu-shot-vaccine/11371175/
https://abc7news.com/covid-test-to-stay-testing-near-me-how-quickly-after-exposure-is-contagious-schools/11352235/
https://abc7news.com/sfusd-

In [36]:
#for result in results:
#    print(result.get_property('href'))
print(results[-1].get_property('href'))
page_driver = webdriver.Firefox()
page_driver.get(results[-1].get_property('href'))

    

https://abc7news.com/oakland-school-board-george-floyd-resolution-ok-program-unified-district/11178201/


In [37]:
headline = page_driver.find_elements_by_class_name("headline")[0].text
story_time = page_driver.find_elements_by_tag_name("time")[0].text
body_text = page_driver.find_elements_by_class_name("body-text")[0].text

print(headline)
print(story_time)

page_driver.quit()


Oakland school board brings back mentorship program after community outcry
Thursday, October 28, 2021


strptime: https://www.tutorialspoint.com/python/time_strptime.htm

In [47]:

print(story_time)
#//*** Convert Story Time to datetime
story_date = datetime.datetime.strptime(story_time, "%A, %B %d, %Y")
print(story_date)

Thursday, October 28, 2021
2021-10-28 00:00:00


2021-10-01 00:00:00
2021-10-28 00:00:00
False


In [6]:
driver.quit()

In [129]:
"""
##///**** On the Off hand we'll yse requests and beautiful soup
#//*** It's a good reference anyway

import requests
raw_html = requests.get(results[-1].get_property('href'))

print(raw_html.text)

from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_html.text, 'html.parser')
#soup = BeautifulSoup(raw_html.text, 'html5lib')
headline = soup.find_all(class_="headline")[0].text

print(headline)

soup.find_all(class_="meta lastmodified")[0]
soup.prettify()
"""
print()